In [ ]:
import os
import re

import benchlingapi
import Bio.Restriction as Restriction
import pandas as pd
import pygsheets
import RNA
import toml
from Bio.Seq import Seq
from tqdm.auto import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.commands.parser as cmd_parser
import paulssonlab.cloning.commands.semantics as cmd_semantics
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.util as cloning_util
import paulssonlab.cloning.workflow as workflow
from paulssonlab.api.util import base_url

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

In [ ]:
# FROM: https://github.com/ViennaRNA/ViennaRNA/issues/64

# read DNA parameters
RNA.read_parameter_file(
    f"{os.environ['CONDA_PREFIX']}/share/ViennaRNA/dna_mathews2004.par"
)


def primer_secondary_structure(seq):
    md = RNA.md()
    fc = RNA.fold_compound(seq, md)
    (_, mfe_monomer) = fc.mfe()
    fc_dimer = RNA.fold_compound(f"{seq}&{seq}", md)
    (_, mfe_homodimer) = fc_dimer.mfe()
    return mfe_monomer, mfe_homodimer

# Flipped backbone primers

In [ ]:
nonflipped_r = reg.get("oLIB193")["_seq"]
nonflipped_f = reg.get("oLIB194")["_seq"]
uns1 = reg.get("UNS1")["_seq"]
uns10 = reg.get("UNS10")["_seq"]
bb_prefix = reg.get("BioBrick_prefix")["_seq"]
bb_suffix = reg.get("BioBrick_suffix")["_seq"]

In [ ]:
binding_r = nonflipped_r[len(uns1) :]
binding_f = nonflipped_f[len(uns10) :]

In [ ]:
nonflipped_r[: len(uns1)] == uns1.reverse_complement().seq

In [ ]:
nonflipped_f[: len(uns10)] == uns10.seq

In [ ]:
uns1.reverse_complement() + binding_r

In [ ]:
nonflipped_r == sequence.get_seq(uns1.reverse_complement() + binding_r)

In [ ]:
nonflipped_f == sequence.get_seq(uns10 + binding_f)

In [ ]:
uns1.reverse_complement().seq

In [ ]:
uns10.seq

In [ ]:
# good for 174/5
# new_binding_r = Seq("ctgcaaaactccttaattaaag")
# new_binding_f = Seq("gacttcgctctagtc")

new_binding_r = Seq("caaaactccttaattaaagg")
new_binding_f = Seq("cttcgctctagtcttg")

In [ ]:
flipped_oldsite_r = uns10 + binding_r
flipped_oldsite_f = uns1.reverse_complement() + binding_f
new_nonflipped_r = uns1.reverse_complement() + new_binding_r  # [:-1]
new_nonflipped_f = uns10 + new_binding_f
flipped_r = uns10 + new_binding_r
flipped_f = uns1.reverse_complement() + new_binding_f  # [:-1]

In [ ]:
primers = {
    "Name": "Bases",
    # "oLIB193": nonflipped_r,
    # "oLIB194": nonflipped_f,
    "oLIB173": flipped_oldsite_r,
    "oLIB174": flipped_oldsite_f,
    "oLIB175": new_nonflipped_r,
    "oLIB176": new_nonflipped_f,
    "oLIB177": flipped_r,
    "oLIB178": flipped_f,
}
primers = {n: str(sequence.get_seq(p)) for n, p in primers.items()}
for name, seq in primers.items():
    print(f"{name}\t{seq}")

In [ ]:
for name, seq in primers.items():
    if name == "Name":
        continue
    mfe_monomer, mfe_homodimer = primer_secondary_structure(seq)
    print(f"{name}\t{mfe_monomer}\t{mfe_homodimer}")

In [ ]:
import primer3

In [ ]:
primer3.calcTm(new_binding_r.encode())

In [ ]:
# test gibson for 3G

In [ ]:
# single function for syncing a benchling folder of (and optionally attaching primers to sequences?)